# Terminal Processing

© Crown-owned copyright 2024, Defence Science and Technology Laboratory UK

## Simulation Layer Implementation.

This notebook serves as a guide on the functionality and use of the new Terminal simulation component.

The Terminal service comes pre-installed on most Nodes (The exception being Switches, as these are currently dumb). 

In [ ]:
from primaite.simulator.system.services.terminal.terminal import Terminal
from primaite.simulator.network.container import Network
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.system.applications.red_applications.ransomware_script import RansomwareScript
from primaite.simulator.system.services.terminal.terminal import RemoteTerminalConnection

def basic_network() -> Network:
    """Utility function for creating a default network to demonstrate Terminal functionality"""
    network = Network()
    node_a = Computer(hostname="node_a", ip_address="192.168.0.10", subnet_mask="255.255.255.0", start_up_duration=0)
    node_a.power_on()
    node_b = Computer(hostname="node_b", ip_address="192.168.0.11", subnet_mask="255.255.255.0", start_up_duration=0)
    node_b.power_on()
    network.connect(node_a.network_interface[1], node_b.network_interface[1])
    return network

The terminal can be accessed from a `Node` via the `software_manager` as demonstrated below. 

In the example, we have a basic network consisting of two computers, connected to form a basic network.

In [ ]:
network: Network = basic_network()
computer_a: Computer = network.get_node_by_hostname("node_a")
terminal_a: Terminal = computer_a.software_manager.software.get("Terminal")
computer_b: Computer = network.get_node_by_hostname("node_b")
terminal_b: Terminal = computer_b.software_manager.software.get("Terminal")

To be able to send commands from `node_a` to `node_b`, you will need to `login` to `node_b` first, using valid user credentials. In the example below, we are remotely logging in to the 'admin' account on `node_b`, from `node_a`. 
If you are not logged in, any commands sent will be rejected by the remote.

Remote Logins return a RemoteTerminalConnection object, which can be used for sending commands to the remote node. 

In [ ]:
# Login to the remote (node_b) from local (node_a)
term_a_term_b_remote_connection: RemoteTerminalConnection = terminal_a.login(username="admin", password="admin", ip_address="192.168.0.11")

You can view all active connections to a terminal through use of the `show()` method.

In [ ]:
terminal_b.show()

The new connection object allows us to forward commands to be executed on the target node. The example below demonstrates how you can remotely install an application on the target node.

In [ ]:
term_a_term_b_remote_connection.execute(["software_manager", "application", "install", "RansomwareScript"])

In [ ]:
computer_b.software_manager.show()

The code block below demonstrates how the Terminal class allows the user of `terminal_a`, on `computer_a`, to send a command to `computer_b` to create a downloads folder. 


In [ ]:
# Display the current state of the file system on computer_b
computer_b.file_system.show()

# Send command
term_a_term_b_remote_connection.execute(["file_system", "create", "folder", "downloads"])

The resultant call to `computer_b.file_system.show()` shows that the new folder has been created.

In [ ]:
computer_b.file_system.show()

When finished, the connection can be closed by calling the `disconnect` function of the Remote Client object

In [ ]:
# Display active connection
terminal_a.show()
terminal_b.show()

term_a_term_b_remote_connection.disconnect()

terminal_a.show()
terminal_b.show()

Disconnected Terminal sessions will no longer show in the node's Terminal connection list, but will be under the historic sessions in the `user_session_manager`.

In [ ]:
computer_b.user_session_manager.show(include_historic=True, include_session_id=True)

## Game Layer Implementation

This notebook section will detail the implementation of how the game layer utilises the terminal to support different agent actions.

The ``Terminal`` is used in a variety of different ways in the game layer. Specifically, the terminal is leveraged to implement the following actions:


|  Game Layer Action                | Simulation Layer         |
|-----------------------------------|--------------------------|
| ``NODE_SEND_LOCAL_COMMAND``       | Uses the given user credentials, creates a ``LocalTerminalSession`` and executes the given command and returns the ``RequestResponse``.
| ``SSH_TO_REMOTE``                 | Uses the given user credentials and remote IP to create a ``RemoteTerminalSession``.
| ``NODE_SEND_REMOTE_COMMAND``      | Uses the given remote IP to locate the correct ``RemoteTerminalSession``, executes the given command and returns the ``RequestsResponse``.

### Game Layer Setup

Similar to other notebooks, the next code cells create a custom proxy agent to demonstrate how these commands can be leveraged by agents in the ``UC2`` network environment.

If you're unfamiliar with ``UC2`` then please refer to the [UC2-E2E-Demo notebook for further reference](./Data-Manipulation-E2E-Demonstration.ipynb).

In [ ]:
import yaml
from primaite.config.load import data_manipulation_config_path
from primaite.session.environment import PrimaiteGymEnv

In [ ]:
custom_terminal_agent = """
  - ref: CustomC2Agent
    team: RED
    type: ProxyAgent
    observation_space: null
    action_space:
      action_list:
        - type: DONOTHING
        - type: NODE_SEND_LOCAL_COMMAND
        - type: SSH_TO_REMOTE
        - type: NODE_SEND_REMOTE_COMMAND
      options:
        nodes:
          - node_name: client_1
        max_folders_per_node: 1
        max_files_per_folder: 1
        max_services_per_node: 2
        max_nics_per_node: 8
        max_acl_rules: 10
        ip_list:
          - 192.168.1.21
          - 192.168.1.14
        wildcard_list:
          - 0.0.0.1
      action_map:
        0:
          action: DONOTHING
          options: {}
        1:
          action: NODE_SEND_LOCAL_COMMAND
          options:
            node_id: 0
            username: admin
            password: admin
            command:
                - file_system
                - create
                - file
                - downloads
                - dog.png
                - False
        2:
          action: SSH_TO_REMOTE
          options:
            node_id: 0
            username: admin
            password: admin
            remote_ip: 192.168.10.22
        3:
          action: NODE_SEND_REMOTE_COMMAND
          options:
            node_id: 0
            remote_ip: 192.168.10.22
            command:
                - file_system
                - create
                - file
                - downloads
                - cat.png
                - False
    reward_function:
      reward_components:
        - type: DUMMY
"""
custom_terminal_agent_yaml = yaml.safe_load(custom_terminal_agent)

In [ ]:
with open(data_manipulation_config_path()) as f:
    cfg = yaml.safe_load(f)
    # removing all agents & adding the custom agent.
    cfg['agents'] = {}
    cfg['agents'] = custom_terminal_agent_yaml
    
env = PrimaiteGymEnv(env_config=cfg)

client_1: Computer = env.game.simulation.network.get_node_by_hostname("client_1")
client_2: Computer = env.game.simulation.network.get_node_by_hostname("client_2")

### Terminal Action | ``NODE_SEND_LOCAL_COMMAND`` 

The yaml snippet below shows all the relevant agent options for this action:

```yaml

    action_space:
      action_list:
      ...
        - type: NODE_SEND_LOCAL_COMMAND
      ...
      options:
        nodes: # Node List
          - node_name: client_1
        ...
    ...
      action_map:
        1:
          action: NODE_SEND_LOCAL_COMMAND
          options:
            node_id: 0 # Index 0 at the node list.
            username: admin
            password: admin
            command:
                - file_system
                - create
                - file
                - downloads
                - dog.png
                - False
```

In [ ]:
env.step(1)
client_1.file_system.show(full=True)

### Terminal Action | ``SSH_TO_REMOTE``  

The yaml snippet below shows all the relevant agent options for this action:

```yaml

    action_space:
      action_list:
      ...
        - type: SSH_TO_REMOTE
      ...
      options:
        nodes: # Node List
          - node_name: client_1
        ...
    ...
      action_map:
        2:
          action: SSH_TO_REMOTE
          options:
            node_id: 0 # Index 0 at the node list.
            username: admin
            password: admin
            remote_ip: 192.168.10.22 # client_2's ip address.
```

In [ ]:
env.step(2)
client_2.session_manager.show()

### Terminal Action |  ``NODE_SEND_REMOTE_COMMAND``

The yaml snippet below shows all the relevant agent options for this action:

```yaml

    action_space:
      action_list:
      ...
        - type: NODE_SEND_REMOTE_COMMAND
      ...
      options:
        nodes: # Node List
          - node_name: client_1
        ...
    ...
      action_map:
        1:
          action: NODE_SEND_REMOTE_COMMAND
          options:
            node_id: 0 # Index 0 at the node list.
            remote_ip: 192.168.10.22
            commands:
                - file_system
                - create
                - file
                - downloads
                - cat.png
                - False
```

In [ ]:
env.step(3)
client_2.file_system.show(full=True)